In [3]:
import cv2

In [4]:
cap = cv2.VideoCapture(0)  

In [5]:
import boto3

In [6]:
INSTANCE_AMI="ami-0a2acf24c0d86e927"
INSTANCE_TYPE="t2.micro"
INSTANCE_COUNT=1
RESOURCE_TYPE="ec2"
REGION="ap-south-1"
SECURITY_GROUP_ID="sg-07c97700945cbdfa0"

In [7]:
ec2 = boto3.resource(RESOURCE_TYPE,REGION)

In [20]:
allOs=[]

In [9]:
def launchEC2Instance():
    instance = ec2.create_instances(
        ImageId=INSTANCE_AMI,
        MinCount=INSTANCE_COUNT,
        MaxCount=INSTANCE_COUNT,
        InstanceType=INSTANCE_TYPE,
        SecurityGroupIds=[SECURITY_GROUP_ID]
    )
    instance_id = instance[0].id
    allOs.append(instance_id)
    print(f"Instance Id of started ec2 instance: {instance_id}",end="\n")
    print(f"Total number instances launched: {len(allOs)}",end="\n")
    return instance_id

In [10]:
def removeEC2Instance():
    osremove = allOs.pop()
    ec2.instances.filter(InstanceIds=[osremove]).terminate()
    print(f"Instance Id deleted: {osremove}",end="\n")
    print(f"Total number of os: {len(allOs)}",end="\n")

In [11]:
#  pip install cvzone, mediapipe
from cvzone.HandTrackingModule import HandDetector

In [12]:
detector = HandDetector(maxHands=1)

In [13]:

def detectHand(image, draw=False):
    detector = HandDetector(maxHands=1)
    hand = detector.findHands(image, draw=draw)
    if hand:
        lmList = hand[0]
        totalFinger = detector.fingersUp(lmList)
        if totalFinger == [0,1,1,0,0]:
            print("Two fingers detected.", end="\n")
            launchEC2Instance()
        elif totalFinger == [0,1,0,0,0]:
            print("One fingers detected.", end="\n")
            removeEC2Instance()
        else:
            print("No fingers detected.", end="\n")
        return totalFinger
    
  

In [14]:
def liveDetector():
    while True:
        status, photo = cap.read()

        detectHand(photo)

        cv2.imshow("my photo", photo)
    
        if cv2.waitKey(1000) == 13:
            break
    cv2.destroyAllWindows()

In [19]:
# Only after the all the work is done
# cap.release()

In [16]:
liveDetector()

No fingers detected.
